In [1]:
from pybliometrics.scopus import AbstractRetrieval, AuthorRetrieval, SerialTitle, ScopusSearch
from tqdm import tqdm
import pandas as pd
import sqlite3
pd.set_option('max.columns', None)

In [3]:
# 수집한 객체에서 각각 필요한 데이터를 리스트로 출력하여 dataframe에 넣을 준비
def get_abstract_data(obj):
    eid = obj.eid
    title = obj.title # Final
    abstract = obj.abstract # Final
    author_keywords = "; ".join(obj.authkeywords or "") # Final
    index_keywords = "; ".join(obj.idxterms or "")
    published_year = obj.coverDate[:4] # Final
    language = obj.language # 영어 기반의 버트 모델이므로 영어 이외의 문서는 제거
    affiliation_ids = "; ".join([str(affiliation_info.id) for affiliation_info in obj.affiliation or ""]) # 소속 기관의 연구자 수, 총 논문 수를 구할 ID
    author_ids = "; ".join([str(author_info.auid) for author_info in obj.authors or ""])
    journal_issn = obj.issn
    refer_ids = "; ".join([ref.id for ref in obj.references or "" if ref.id])
    funding_cnt = len(obj.funding or [])

    return [eid, title, abstract, author_keywords, index_keywords, published_year, language, affiliation_ids, author_ids, refer_ids, journal_issn, funding_cnt]

def get_author_data(auid):
    au = AuthorRetrieval(auid)
    citaion_count = au.citation_count or 0
    cited_by_count = au.cited_by_count or 0
    h_index = au.h_index or 0
    research_start_year = au.publication_range[0] if au.publication_range else None

    return [auid, citaion_count, cited_by_count, h_index, research_start_year]

def get_sjr_data(issn):
    # issn 데이터 중 동저널 다른 issn을 가진 데이터가 있음 => '00000000 00000000' 띄어쓰기로 구분된 문자열 형태로 자료가 삽입되어 있음
    issn_list = issn.split(' ')
    for issn in issn_list:
        try:
            st = SerialTitle(issn, view='ENHANCED', years="2004-2010")
            issn_sjr_df = pd.DataFrame(st.sjrlist, columns=['year', 'sjr'])
            issn_sjr_df.insert(0, 'issn', issn)
            return issn_sjr_df
        except Exception as e:
            print(issn, '데이터 접근 오류 =>', e)
            pass

def load_table(table_name, db_file_path='rsc/training_data/X_data.db'):
    """
    데이터베이스 테이블을 가져오는 함수
        파라미터
            table_name [str] : 테이블 이름
            db_file_path [str] : 테이블이 들어있는 데이터베이스 파일 경로
        리턴값
            table [DataFrame] : 데이터프레임
    """
    query = 'SELECT * FROM {}'.format(table_name)
    with sqlite3.connect(db_file_path) as conn:
        table = pd.read_sql(query, conn)
    
    return table

def make_query_by_eids(eids:list):
    """
    EID 리스트를 넣으면 ScopusSearch(query=eid( {입력} ))에 맞게 데이터를 가공하여 출력받는 함수
    --- 파라미터 ---
    eids : 리스트 형태
    --- 출력 ---
    str : 쿼리
    """
    eid_list = ['"2-s2.0-' + str(eid) + '"' for eid in eids]
    return "eid ( " + " OR ".join(eid_list) + " )"

def filter_refer_ids(id_list, filename):
    """
    저장된 References DB와 비교하여 이미 수집한 데이터는 지우고, 수집해야할 데이터만 필터
        파라미터
        - id_list : 검사할 ID 리스트
        - filename : 비교할 DB 파일 경로
        리턴
        - list : 수집해야할 ID 리스트
    """
    import sqlite3
    import pandas as pd

    conn = sqlite3.connect(filename)
    df = pd.read_sql("select * from refer_table", conn)
    return list(set(id_list) - set(df['eid'])) # id_list 집합에서 DB 파일에 있는 eid 집합을 뺀다.


# 년도별 저널 SJR 데이터

In [ ]:
# X_data 불러와서 수집할 저널 ISSN 정보를 추출
conn = sqlite3.connect('rsc/training_data/X_data.db')
query = 'SELECT * FROM ABSTRACT_RETRIEVAL_API where language == "eng"'
X_data = pd.read_sql(query, conn)
conn.close()

ISSNs = X_data.journal_issn.unique()

issn_year_sjr_df = pd.DataFrame()

for issn in tqdm(ISSNs):
    try:
        retrieval_df = get_sjr_data(issn)
        issn_year_sjr_df = issn_year_sjr_df.append(retrieval_df)
    except Exception as e:
        print(issn)
        print(e)
        break

In [ ]:
with sqlite3.connect('rsc/training_data/X_data.db') as conn:
    issn_year_sjr_df.to_sql('JOURNAL_SJR', conn, if_exists='append', index=False)

# 논문 서지사항 정보

In [ ]:
# X 데이터 수집을 위한 기준 데이터 생성
with open("rsc/preparation_data/AI_target_eids_2005-2010.txt", "r") as f:
     ai_paper_eids = [eid.strip() for eid in f.readlines()]
print("AI 논문 개수:", len(ai_paper_eids))

with open("rsc/preparation_data/automotive_target_eids_2005-2010.txt", "r") as f:
     ae_paper_eids = [eid.strip() for eid in f.readlines()]
print("AE 논문 개수:", len(ae_paper_eids))

df_ai = pd.DataFrame({'eid':ai_paper_eids})
df_ai['div'] = 'AI'
df_ae = pd.DataFrame({'eid':ae_paper_eids})
df_ae['div'] = 'AE'

df = pd.concat([df_ai, df_ae]).reset_index(drop=True)

In [ ]:
try: 
    conn = sqlite3.connect("x_temp_abstract_retrieval.db")
    abstract = pd.read_sql("select * from abstract_retrieval", conn)
    conn.close()
    eids_for_retrieval = set(df['eid']) - set(abstract['eid'])
except:
    eids_for_retrieval = df['eid'].tolist()

ab_objs = []
for eid in tqdm(eids_for_retrieval):
    try:
        ab = AbstractRetrieval(eid, view='FULL')
        ab_objs.append(ab)
    except Exception as e:
        print(e)

In [ ]:
abstract_datas = []
for ab_obj in tqdm(ab_objs):
    abstract_datas.append(get_abstract_data(ab_obj))

columns = ['eid', 'title', 'abstract', 'author_keywords', 'index_keywords', 'published_year', 'language', 'affiliation_ids', 'author_ids', 'refer_ids', 'journal_issn', 'funding_cnt'] 
abstract = pd.DataFrame(abstract_datas, columns=columns)
conn = sqlite3.connect("rsc/training_data/X_abstract_retrieval.db")
abstract.to_sql('ABSTRACT_RETRIEVAL_API', conn, if_exists='append', index=False)
conn.close()

# 저자 관련 데이터 수집

In [ ]:
conn = sqlite3.connect('rsc/training_data/X_data.db')
query = 'SELECT * FROM ABSTRACT_RETRIEVAL_API where language == "eng"'
X_data = pd.read_sql(query, conn)
conn.close()

In [ ]:
# 저자 관련 데이터 수집 (Author ciations, Author publications, Author H-index) -> 총 3개
auids = list(set([auid.strip() for auid in "; ".join(X_data['author_ids']).split("; ") if auid]))

# 기존에 수집하여 데이터베이스에 저장한 auid는 제외
with sqlite3.connect('rsc/training_data/X_data.db') as conn:
    data = pd.read_sql('select * from AUTHOR_RETRIEVAL_API', conn)
auids = list(set(auids) - set(data['auid'].astype(str)))

print('수집할 저자 관련 데이터 수: {:,}개'.format(len(auids)))

for auid in tqdm(auids):
    author_data = get_author_data(auid)
    with sqlite3.connect('rsc/training_data/X_data.db') as conn:
        cursor = conn.cursor()
        query = "insert into AUTHOR_RETRIEVAL_API values (?, ?, ?, ?, ?)"
        cursor.execute(query, author_data)
        conn.commit()

# 참고문헌 데이터 제목 수집

In [4]:
abstract_info = load_table("ABSTRACT_RETRIEVAL_API",)

# 영어로된 데이터만 추출
abstract_info = abstract_info[abstract_info['language'] == 'eng'].reset_index(drop=True)

In [8]:
# 학습 데이터의 참고문헌 제목을 사용하기 위해 데이터 수집
DB_FILE_PATH = "./rsc/training_data/reference.db"

refer_ids = [refer_id.strip() for refer_id in "; ".join(abstract_info['refer_ids']).split("; ") if refer_id != ""]
try:
    refer_ids = filter_refer_ids(refer_ids, DB_FILE_PATH)
except:
    pass

for i in tqdm(range((len(refer_ids)//100))):
    query = make_query_by_eids(refer_ids[100*i:100*(i+1)])
    s = ScopusSearch(query=query, download=True)
    refer_df = pd.DataFrame(s.results)
    refer_df = refer_df[['eid', 'title']]

    conn = sqlite3.connect(DB_FILE_PATH)
    refer_df.to_sql("refer_table", conn, if_exists='append', index=False)
    conn.close()

 47%|████▋     | 4036/8605 [3:22:50<3:42:57,  2.93s/it] 

Scopus429Error: Quota Exceeded

# 데이터 전처리
### 테이블명
- ABSTRACT_RETRIEVAL_API
- AUTHOR_RETRIEVAL_API
- JOURNAL_SJR

In [8]:
# SQL로 데이터 불러오기
journal_sjr = load_table("JOURNAL_SJR")
journal_sjr.drop_duplicates(inplace=True) # 중복 제거
author_info = load_table("AUTHOR_RETRIEVAL_API")
abstract_info = load_table("ABSTRACT_RETRIEVAL_API")

# 영어로된 데이터만 추출
abstract_info = abstract_info[abstract_info['language'] == 'eng'].reset_index(drop=True)

# Journal_ISSN이 스페이스를 구분으로 2개씩 붙어있는 것들이 있으므로 컬럼을 2개로 나눈다.
abstract_info = pd.concat([abstract_info, abstract_info['journal_issn'].str.split(" ", expand=True).rename(columns={0:"issn1", 1:"issn2"})], axis=1)


# AI, AE 라벨 붙이기 
eid_label_dict = {}

with open("./rsc/preparation_data/automotive_target_eids_2005-2010.txt", 'r') as f:
    ae_eids = f.readlines()
ae_eids = {ae_eid.strip():'AE' for ae_eid in ae_eids}

with open("./rsc/preparation_data/AI_target_eids_2005-2010.txt", 'r') as f:
    ai_eids = f.readlines()
ai_eids = {ai_eid.strip():'AI' for ai_eid in ai_eids}

eid_label_dict.update(ae_eids)
eid_label_dict.update(ai_eids)

abstract_info.insert(1, 'field', abstract_info['eid'].map(eid_label_dict))

##### (4) Journal Impact Factor(JIF) #####
# JOIN 하기 위해 데이터의 타입과 컬럼명 변경 작업 진행
abstract_info = abstract_info.astype(str)
journal_sjr = journal_sjr.astype(str)

# 데이터 합치기
x_temp = pd.merge(abstract_info, journal_sjr, how='left', left_on=['issn1', 'published_year'], right_on=['issn', 'year'])
abstract_info = pd.merge(x_temp, journal_sjr, how='left', left_on=['issn2', 'published_year'], right_on=['issn', 'year'])

# 데이터 합친 후 필요 없는 컬럼 제거
abstract_info = abstract_info.drop(labels=['journal_issn', 'issn_x', 'year_x', 'issn_y', 'year_y'], axis=1)

# Bibliometric factor 가공
abstract_info['sjr_x'] = abstract_info['sjr_x'].fillna("")
abstract_info['sjr_x'] = abstract_info.apply(lambda x: x['sjr_x'] if x['sjr_x'] else x['sjr_y'], axis=1)
abstract_info = abstract_info.drop(labels=['sjr_y'], axis=1).rename(columns={'sjr_x':'sjr'})
###########################################

##### (6) 저자 수  #####
abstract_info['number_of_authors'] = abstract_info['author_ids'].str.split(";").str.len()
###########################################

##### (7) Number of institutions #####
abstract_info['number_of_institutions'] = abstract_info['affiliation_ids'].str.split(";").str.len()
###########################################

##### (8) Number of references #####
abstract_info['number_of_references'] = abstract_info['refer_ids'].str.split(";").str.len()
###########################################


In [1]:
abstract_info

NameError: name 'abstract_info' is not defined

# BERT 사전학습 언어모델 사용

In [ ]:
from keras_bert import load_trained_model_from_checkpoint, extract_embeddings

In [ ]:
config_path = "./engi_bert_L-12_H-768_A-12/bert_config.json"
ckpt_path = "./engi_bert_L-12_H-768_A-12/bert_model.ckpt"

In [ ]:
model = load_trained_model_from_checkpoint(
    config_path,
    ckpt_path,
    training=False,
    trainable=False,
    seq_len=128
)

In [ ]:
model.summary()

In [ ]:
paths

In [ ]:
from keras_bert import extract_embeddings

model_path = 'engi_bert_L-12_H-768_A-12'
texts = [
    ('automotive is a car.'),
    ('A car i automotive.'),
]

embeddings = extract_embeddings(model_path, texts)

In [ ]:
embeddings[1].shape

In [ ]:
token_dict = {}
with open('./engi_bert_L-12_H-768_A-12/vocab.txt', 'r') as f:
    tokens = f.readlines()

token_list = [token.strip() for token in tokens]

for idx, value in enumerate(token_list):
    token_dict[value] = idx

In [ ]:
from keras_bert import Tokenizer

tokenizer = Tokenizer(token_dict)
tokenizer.tokenize(t1)

In [ ]:
t1

In [ ]:
from transformers import BertForMaskedLM

In [ ]:
from transformers import BertTokenizer, TFAutoModel

tokenizer = BertTokenizer.from_pretrained("./engi_bert_L-12_H-768_A-12")

In [ ]:
tokenizer.tokenize('automotive is a car.')

In [ ]:
# https://www.kaggle.com/hiromoon166/load-bert-fine-tuning-model 참고
from keras_bert import load_trained_model_from_checkpoint
from tensorflow.keras.utils import plot_model

config_file = "./engi_bert_L-12_H-768_A-12/bert_config.json"
ckpt_file = "./engi_bert_L-12_H-768_A-12/bert_model.ckpt"

model = load_trained_model_from_checkpoint(config_file, ckpt_file, training=False, seq_len=512)
# model.summary()
# plot_model(model)

In [ ]:
model.trainable_variables